## Importamos librerias necesarias

In [2]:
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd
from datetime import datetime, time
import time
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
warnings.filterwarnings('ignore')


In [3]:
from skimage.feature import hog
from skimage.color import rgb2gray
from skimage.feature import local_binary_pattern

## Funcion Gaze para el calculo de rotacion de la cabeza

In [4]:
def gazes(image,results):
    img_h, img_w, img_c = image.shape
    face_3d = []
    face_2d = []
    for idx, lm in enumerate(results):
        if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
            if idx == 1:
                nose_2d = (lm.x * img_w, lm.y * img_h)
                nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)

            x, y = int(lm.x * img_w), int(lm.y * img_h)

            # Get the 2D Coordinates
            face_2d.append([x, y])

            # Get the 3D Coordinates
            face_3d.append([x, y, lm.z])       

    # Convert it to the NumPy array
    face_2d = np.array(face_2d, dtype=np.float64)

    # Convert it to the NumPy array
    face_3d = np.array(face_3d, dtype=np.float64)

    # The camera matrix
    focal_length = 1 * img_w

    cam_matrix = np.array([ [focal_length, 0, img_h / 2],
                            [0, focal_length, img_w / 2],
                            [0, 0, 1]])

    # The distortion parameters
    dist_matrix = np.zeros((4, 1), dtype=np.float64)

    # Solve PnP
    success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

    # Get rotational matrix
    rmat, jac = cv2.Rodrigues(rot_vec)

    # Get angles
    angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

    # Get the y rotation degree
    x = angles[0] * 360
    y = angles[1] * 360
    z = angles[2] * 360


    # See where the user's head tilting
    if y < -10:
        text = [0,0,1,0,0]
    elif y > 10:
        text = [0,0,0,1,0]
    elif x < -10:
        text = [0,1,0,0,0]
    elif x > 10:
        text = [0,0,0,0,1]
    else:
        text = [1,0,0,0,0]

    gaze_row = list(np.array([x, y ,z]))+list(text)
    gaze_column = list(np.array(["g_x", "g_y", "g_z","Forward","Looking Down","Looking Left","Looking Right","Looking Up"]))
    
    return gaze_row, gaze_column

## Funciones para extraer las AU

In [5]:
def proyeccion(image,mascara,results):
    x = []
    y=[]
    X=[]
    Y=[]
    height, width, _ = image.shape
    #Inicializamos el método con imagenes en este caso van a ser fijas para extraer los entrenamientos
    if results.face_landmarks is not None:
        for index in results.face_landmarks.landmark:
            x.append(index.x*width) 
            y.append(index.y*height)
        for index in mascara:
            X.append(x[index])
            Y.append(y[index])
    return X,Y
def procesamiento(img,results):
    mascara_centro_ojos=[159,145,33,133,386,374,362,263] #Máscara para conseguir el punto central del ojo
    mascara_cara_superior=[177]
    mascara_cara_inferior=[227]
    mascara_recorte=[10,152,454,234]
    
    #ROTACIÓN BASADA EN OJOS
    x,y=proyeccion(img,mascara_centro_ojos,results) #Nos devuelve los puntos en la imagen con face mesh
    ojo_izquierdo=[(x[0]+x[1])/2,(y[2]+y[3])/2] #Conseguimos el punto central de los ojos izquierdo y derecho
    ojo_derecho=[(x[4]+x[5])/2,(y[6]+y[7])/2]
    #print('La mascara ojos funciona correctamente')
    #Calculamos la pendiente y ángulo entre los puntos centrales para enderezar la imagen
    pendiente= (ojo_derecho[1]-ojo_izquierdo[1])/(ojo_derecho[0]-ojo_izquierdo[0])
    angulo=(180/math.pi)*math.atan(pendiente)
    rows, cols = img.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angulo, 1)
    img_rotated = cv2.warpAffine(img, M, (cols,rows)) #Aquí ya tenemos la imagen rotada
    
      #RECORTE CARA BASADA EN 4 PUNTOS
    x,y=proyeccion(img_rotated,mascara_recorte,results)
    recorte_x_derecha=np.max(x)
    recorte_x_izquierda=np.min(x)
    recorte_y_arriba=np.min(y)
    recorte_y_abajo=np.max(y)
    nueva_imagen=img_rotated[int(recorte_y_arriba):int(recorte_y_abajo),int(recorte_x_izquierda):int(recorte_x_derecha)]
    
    #Normalizamos la imagen en tamaño 
    nueva_imagen=cv2.resize(nueva_imagen, (100, 100), interpolation=cv2.INTER_LINEAR)
    return nueva_imagen
def modulo_visualizacion_de_resultados(img,Predictores,pca_pipe_1,pca_pipe_2):
    
    #Recibo la imagen y le saco las HOGs
    feature_1=pd.DataFrame(hog(img, orientations=8, pixels_per_cell=(15, 15), cells_per_block=(2, 2),block_norm='L2-Hys', visualize=False, transform_sqrt=False,feature_vector=True,channel_axis=-1))
   
    #Recibo la imagen y le saco las LBP
    image=rgb2gray(img)
    lbp = local_binary_pattern(image, 8, 4, 'default')
    values, bins = np.histogram(lbp, bins=np.arange(257))
    feature_2=pd.DataFrame(values)
    feature_2=feature_2.apply(lambda x: np.log(x+1))
    
    #Reduzco la dimensionalidad
    feature_1=pca_pipe_1.transform(feature_1.T)[0][0:6*19]
    feature_2=pca_pipe_2.transform(feature_2.T)[0][0:6*26]
    df1=pd.concat([pd.DataFrame(feature_1).T,pd.DataFrame(feature_2).T],axis=1)
    Potencia=np.array([math.floor(10*(x.predict_proba([df1.values[0]])[0][1])) for x in Predictores])
    return Potencia

## Carga de modelos predictores para AU

In [6]:
with open("AU_modelos_listados por orden.pickle", "rb") as f:
    lista_AU = pickle.load(f)
with open("au_predictores.pickle", "rb") as f:
    predictores = pickle.load(f)
with open("pca_pipe1.pickle", "rb") as f:
    pca_pipe_1 = pickle.load(f)
with open("pca_pipe2.pickle", "rb") as f:
    pca_pipe_2 = pickle.load(f)

## Script de extracciòn y preprocesamiento de datos para obtencion de archivo .CSV con Face y Pose Landmarks

In [7]:

inicio = time.time()
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
font = cv2.FONT_HERSHEY_SIMPLEX
lista_AU=list(lista_AU)
lista_frame=pd.Series(index = lista_AU)
lista_final=[]
contenido=os.listdir('Train/')
dft=pd.DataFrame()
timeRate = 0.1

listaclip=["ClipID"]
#path=("pruebav.mp4")
for ttv in contenido:
    users = os.listdir('Train/'+ttv+'/')
    for user in users:
        currUser = os.listdir('Train/'+ttv+'/'+user+'/')
        clip = os.listdir('Train/'+ttv+'/'+user+'/')[0]
        path = os.path.abspath('.')+'/Train/'+ttv+'/'+user+'/'   
        try:
            print(clip)
            cap = cv2.VideoCapture(path+clip) #+clip
            FPS = cap.get(5)
            c = 1
            with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5, refine_face_landmarks=True) as holistic:
                while cap.isOpened():
                    success, image = cap.read()
                    if not success:
                        print("Ignoring empty camera frame.")
                    # If loading a video, use 'break' instead of 'continue'.
                        break 
                    #image = cv2.imread('20190622_124921.jpg')
                    # Convert BGR to RGB
                    frameRate = int (FPS) * timeRate
                    if(c % frameRate == 0):
                        results = holistic.process(image)
                        # A CADA FRAME LE HAGO UN PROCESAMIENTO PARA RECORTAR LA CARA CON LAS FUNCIONES PROYECCION Y PREPROCESAMIENTO A PARTIR DE RESULTS DE HOLISTIC
                        IMAGEN_PROCESADA=procesamiento(image,results)
                        #AQUI EXTRAEMOS LAS FEATURES Y SE LAS DAMOS A LOS PREDICTORES
                        potencia=modulo_visualizacion_de_resultados(IMAGEN_PROCESADA,predictores,pca_pipe_1,pca_pipe_2)
                        lista_frame=pd.concat([lista_frame,pd.DataFrame(potencia,index=lista_AU)],axis=1)

                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                        image.flags.writeable = False
                        # Make detections
                        results = holistic.process(image)

                        # Conver back to BGR
                        image.flags.writeable = True
                        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                        # Export coordinate and estimate
                        #try:
                            # Extract gaze
                        gaze = results.face_landmarks.landmark
                        gaze_row, gaze_column= gazes(image,gaze)

                        # Extract pose landmark
                        pose = results.pose_landmarks.landmark
                        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z] #, landmark.visibility
                                                            for landmark in pose]).flatten())
                        pose_colum = list(np.array([["px_" + str(index), "py_" + str(index), "pz_" + str(index)] #, "pv_" + str(index)
                                                    for index, landmark in enumerate(pose)]).flatten())


                        # Extract face landmark
                        face = results.face_landmarks.landmark
                        face_row = list(np.array([[landmark.x, landmark.y, landmark.z] #, landmark.visibility
                                                            for landmark in face]).flatten())

                        face_colum = list(np.array([["fx_" + str(index), "fy_" + str(index), "fz_" + str(index)] #, "fv_" + str(index)
                                                    for index, landmark in enumerate(face)]).flatten())

                        # # Concatenate rows
                        ClipID=[clip]
                        row = pose_row+face_row+gaze_row+ list(potencia)+ ClipID
                        columns_names = pose_colum + face_colum + gaze_column + lista_AU +listaclip

                        X = pd.DataFrame([row])
                        X.columns = columns_names
                        X=X.drop(X.columns[30:99],axis=1)
                        dft=pd.concat([dft,X],ignore_index=True)
                    c += 1

                        #except Exception as e: 
                         #   print(e)               
                          #  pass
        except Exception as e:
            print(e) 
            pass
fin = time.time()
print(fin-inicio)
dft.to_csv ("TestDataset2/"+"pruebaxframe"+".csv", index = False, header=True)

1100011002.avi


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Ignoring empty camera frame.
1100011003.avi
Ignoring empty camera frame.
58.38975238800049
